# CREAR UNA APLICACIÓN SENCILLA

Has entendido las diferentes partes y configuraciones de una aplicación web Flask en las secciones anteriores. Ahora es el momento de escribir la primera. En el siguiente ejemplo, el script de la aplicación define una instancia de aplicación, una única ruta y una única función de vista, como ya hemos dicho. 
Utilizaré Visual Studio Code, que tiene instalada la extensión Python. 
El primer paso es crear una carpeta de proyecto. La mía es firstapp. Nombra la tuya como quieras.

Después de haber cd esa carpeta, crear un entorno virtual. Yo llamaré al mío env.

> python -m venv env

Ahora, escriba el código en el Terminal y ejecútelo. Visual Studio Code se abrirá en una nueva ventana. Ahora, abra la carpeta de la aplicación en la nueva ventana de esta manera:

¿Puedes ver el nombre de tu entorno virtual en la esquina inferior izquierda? El mío tiene "env" como nombre de mi entorno virtual.

Ahora que el entorno virtual está activo, instala Flask en el entorno virtual ejecutando pip install Flask en el Terminal.

Cuando se inicia un símbolo del sistema independiente, ejecute env\Scripts\activar.bat para activar el medio ambiente. Debe comenzar con (env), indicando que está activado.

["miapp.py"](../App/app.py)